In [1]:
#importing the libraries
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")
import nltk
from thefuzz import fuzz
from thefuzz import process

In [5]:
%%time
#loading the data

df=pd.read_csv('wiki_movie_plots_deduped.csv')
df['Genre'].nunique()

Wall time: 1.05 s


2265

In [6]:
# for splitting the multiple genre on the basics of ' ,' and making the new rows
def genre(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page', 'Plot', 'Genre_1','Genre_updated'])
    for i,row in data.iterrows():
        if len(row['Genre_1'].split(','))>1:
            for i in row['Genre_1'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Genre_updated']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Genre_updated']=df['Genre_1']
            df1=pd.concat([df1,df])
    df['Genre_updated']=df['Genre_updated'].apply(lambda x: x.strip())
    return df1.reset_index(drop=True)

In [7]:
#Replacing the unknown with the null string
df['Genre_1']= df['Genre'].str.replace("unknown", "")


In [25]:
#for checking the counts of each genre
dax=df['Genre_1'].value_counts().rename_axis('Genres_1').reset_index(name='counts')
#dax[:40]

In [24]:
#selecting the top 17 genre's
#[i for i in dax['Genres_1'].to_list() if len(i.split())<2][1:18]

In [10]:
#storing the selected top 17 genre's and saving them into a variable choices
choices = [i for i in dax['Genres_1'].to_list() if len(i.split())<2][1:18]


In [11]:
#making the list of unique qenure's and the list containing the list of approximately same genre name 

li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])
        

In [16]:
%%time

#looping over the list of quenres and making the list  of unique qenure's and the list containing the list of approximately same genre name  
for i in range(10,len(dax['Genres_1'].to_list()),10):
    inputt=dax['Genres_1'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)


Wall time: 1min 35s


In [17]:
# removing the extra spaces from the starting and ending
df['Genre_1']=df['Genre_1'].apply(lambda x: x.strip())

In [23]:
#looking into the unique genre and checking the smiliar genre 
#[[row['Genres_1'],row['counts']] for i,row in dax.iterrows() if row['Genres_1'] in choices ]

In [19]:
# making the dict for mapping the names of same genres
replace_dict={
   "science_friction":"sci_fi",
    "sci fi":"sci_fi",
    "science friction":"sci_fi",
    "science fiction":"sci_fi",
    "sci-fi":"sci_fi",
    "amine":"animated",
    "animation":"animated",
    "cartoon":"animated",
    "tokusatsu": "animated",
    "superhero": "animated",
    "suspense": "thriller",
    "dramedy": "drama",
    "blaxploitation": "exploitation",
    "biopic": "biography",
    "bio-pic": "biography",
    "biographical": "biography",
    "kung fu": "martial_arts",
    "martial arts": "martial_arts",
    "kung fu": "martial_arts",
    "kung-fu": "martial_arts",
    "film": "",
    "neo-noir": "noir",
    "devotional": "religious",
    "children": "teen",
    "porno": "adult",
    "history": "historical",
    "rom com": "romance-comedy",
    "rom-com": "romance-comedy",
    "love": "romance",
    "periodic":"historical",
    "period":"historical",
    "football":"sports",
    "youth":'teen',
    "sexual":"adult",
    "/":" ",
    '-':' ',
    "slice of life":'slice_of_life',
    "love":"romance",
}

In [20]:
%%time
#replacing the the genre names
for key,items in replace_dict.items():
    df['Genre_1']= df['Genre_1'].str.replace(key,items)
df['Genre_1']=df['Genre_1'].apply(lambda x: x.strip())

Wall time: 553 ms


In [21]:
%%time
#splitting the multiple genre on the basics of ' ,' and making the new rows
df=genre(df)

Wall time: 2min 9s


In [28]:
# removing the extra space after spliting the multiple genres
df['Genre_updated']=df['Genre_updated'].apply(lambda x: x.strip())

In [29]:
#repeating the same process from taking the top 17 genres and making the list of unique genres once again and list of common genres

dax=df['Genre_updated'].value_counts().rename_axis('Genres_updated').reset_index(name='counts')
choices = [i for i in dax['Genres_updated'].to_list() if len(i.split())<2][1:18]
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])

In [30]:
%%time
# for slecting the single genres
for i in range(10,len(dax['Genres_updated'].to_list()),10):
    inputt=dax['Genres_updated'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)

Wall time: 50.7 s


In [31]:
#making the final list of genres
choices_final=[row['Genres_updated'] for i,row in dax.iterrows() if row['Genres_updated'] in choices if row['counts']>30]

In [32]:
#choices_final

In [33]:
#[[row['Genres_updated'],row['counts']] for i,row in dax.iterrows() if row['Genres_updated'] in choices_final ]
#[row['Genres_updated'] for i,row in dax.iterrows() if row['Genres_updated'] in choices if row['counts']>30]

In [34]:
# for changing the genre according to final list if the genre is single then it will be the same, if it contains the approximatelly same in the choices final list then returning the list of those genres else null string

def genre_list(x):
    if x in choices_final:
        return x
    elif process.extractBests(x, choices_final,score_cutoff=80,limit=20) !=[]:
        #print(choices_final)
        a=process.extractBests(x, choices_final,score_cutoff=80,limit=20)
        #print(a)
        return ','.join([a[i][0] for i in range(len(a))])
    else:
        return ''

In [23]:
#genre_list('drama romance')

'drama,romance'

In [35]:
%%time
# changing the genre according to final list if the genre is single then it will be the same, if it contains the approximatelly same in the choices final list then returning the list of those genres else null string

df['Genre_updated']=df['Genre_updated'].apply(lambda x: genre_list(x))

Wall time: 1min 10s


In [37]:
# for again splitting on the basics of the ','
def genre_updated(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page', 'Plot','Genre_1','Genre_updated','Genre_updated1'])
    for i,row in data.iterrows():
        if len(row['Genre_updated'].split(','))>1:
            for i in row['Genre_updated'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Genre_updated1']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Genre_updated1']=df['Genre_updated']
            df1=pd.concat([df1,df])
    df['Genre_updated1']=df['Genre_updated1'].apply(lambda x: x.strip())       
    return df1.reset_index(drop=True)

In [38]:
%%time
#splitting multiple genres on the basics of ' , '
new_df=genre_updated(df)

Wall time: 2min 33s


In [39]:
# checking the value count of each genre
new_df['Genre_updated1'].value_counts()

drama           9513
comedy          7295
                6950
action          2473
romance         1986
thriller        1924
crime           1604
horror          1593
western          974
musical          960
sci_fi           932
animated         905
adventure        868
family           635
war              604
fantasy          561
mystery          481
biography        457
noir             358
historical       285
short            242
social           223
anime            143
spy              142
martial_arts     133
documentary      130
sports           126
teen             100
serial            83
Name: Genre_updated1, dtype: int64

In [40]:
#saving the updated dataframe into csv
new_df.drop_duplicates().drop(['Genre_1','Genre_updated'],axis=1).rename(columns={'Genre_updated1': 'Genre_updated'}).to_csv('wiki_movie_plots_deduped_cleaned_genre.csv',index=False)